In [1]:
%pip install textblob

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ast
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import nltk
from textblob import TextBlob
from nltk.sentiment import SentimentIntensityAnalyzer

In [3]:
games = pd.read_parquet('data_clean/1-games.parquet')
reviews= pd.read_parquet('data_clean/2-reviews.parquet')
items = pd.read_parquet('data_clean/3-items.parquet')

#### **Analisis del dataframe Reviews**

In [23]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48690 entries, 0 to 48689
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             48690 non-null  object
 1   user_url            48690 non-null  object
 2   reviews_item_id     48690 non-null  int64 
 3   reviews_helpful     48690 non-null  object
 4   reviews_recommend   48690 non-null  bool  
 5   posted_year         48690 non-null  int64 
 6   sentiment_analysis  48690 non-null  int64 
 7   release_year        48690 non-null  int64 
dtypes: bool(1), int64(4), object(3)
memory usage: 2.6+ MB


Se crea una columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [4]:
def sentiment_analysis(review):
    '''
    Ejecuta un análisis de sentimiento en un texto dado y retorna un valor numérico que refleja el sentimiento.
    Se emplea la librería TextBlob para evaluar el sentimiento en un texto dado, asignando un valor numérico basado en la polaridad del sentimiento.

    Parameters:
        review (str): El texto que se someterá a análisis para determinar su sentimiento.

    Returns:
        int: Un valor numérico que indica el sentimiento del texto:
             - 0 para un sentimiento negativo.
             - 1 para un sentimiento neutral o no clasificable.
             - 2 para un sentimiento positivo.
    '''
    if not review or not review.strip():
        return 1

    polarity = TextBlob(review).sentiment.polarity

    if polarity < -0.2:
        return 0
    elif polarity > 0.2:
        return 2
    else:
        return 1


In [ ]:
#cambio el nombre de la columna reviews_review a sentiment_analysis y le aplico la función analisis_sentimiento 
reviews['sentiment_analysis'] = reviews['reviews_review'].apply(sentiment_analysis)
reviews.head()

In [6]:
# Observamos los resultados del análisis de sentimientos
sentiment_count = reviews['sentiment_analysis'].value_counts()
sentiment_count
#lo que mas se repite es el sentimiento positivo

sentiment_analysis
1    29886
2    14629
0     4175
Name: count, dtype: int64

In [7]:
reviews.head()

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,posted_year,sentiment_analysis,release_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011,1,2009
1,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,2011,2,2013
2,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,2013,2,2013
3,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,2014,1,2012
4,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,2014,0,2013


In [ ]:
#eliminamos la columna reviews_review. Nos quedamos con la columna sentiment_analysis para facilitar el trabajo de los modelos (como pide el ennunciado)
reviews = reviews.drop(columns=['reviews_review'])
reviews.columns

In [8]:
reviews.head()

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,posted_year,sentiment_analysis,release_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011,1,2009
1,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,2011,2,2013
2,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,2013,2,2013
3,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,2014,1,2012
4,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,2014,0,2013


#### **Analisis de los dataframes Items y Games** 

In [30]:
items = pd.read_parquet('data_clean/3-items.parquet')

* Se crea un nuevo DataFrame con las columnas 'item_id', 'items_count' y 'item_id'
* Nos va a servir para calcular la funcion de gasto de los usuarios en los items

In [31]:
# Se extraen las columnas 'items_count', 'user_id' e 'item_id'
# items_spend seria la cantidad de veces que un usuario ha comprado un item
# items_count es la cantidad de veces que un usuario ha comprado un item

items_spend = items[['items_count', 'user_id', 'item_id']]
items_spend.head()

,items_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,30
2,277,76561197970982479,300
3,277,76561197970982479,240
4,277,76561197970982479,3830


In [32]:
#creo un nuevo dataframe con los datos de precio y id
game_price = games[['price', 'id']]
game_price = game_price.drop_duplicates(subset='id', keep='first')


In [33]:
# Se renombra 'id' por 'item_id' para unir con items_spend
game_price = game_price.rename(columns={'id':'item_id'})
game_price

,price,item_id
0,4.99,761140
5,0.00,643980
9,0.00,670290
14,0.99,767400
17,3.99,772540
...,...,...
71028,1.99,745400
71032,1.99,773640
71036,4.99,733530
71039,1.99,610660


In [34]:
#uno game_price con items_spend con key: item id
items_spend = items_spend.merge(game_price, on='item_id', how='left')
items_spend


,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,30,4.99
2,277,76561197970982479,300,9.99
3,277,76561197970982479,240,19.99
4,277,76561197970982479,3830,9.99
...,...,...,...,...
3246357,7,76561198329548331,304930,0.00
3246358,7,76561198329548331,227940,0.00
3246359,7,76561198329548331,388490,0.00
3246360,7,76561198329548331,521570,0.00


In [35]:
#verifico si hay valores nulos
items_spend.isna().sum().sort_values(ascending= False)/len(items) * 100

price          15.201786
items_count     0.000000
user_id         0.000000
item_id         0.000000
dtype: float64

In [36]:
items_spend.head()

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,30,4.99
2,277,76561197970982479,300,9.99
3,277,76561197970982479,240,19.99
4,277,76561197970982479,3830,9.99


In [37]:
#fill_precio para rellenar los valores nulos con 0
fill_precio = items_spend['price'].fillna(0.0)

In [38]:
#actualizo el dataframe items_spend concatenando con fill_precio
items_spend = pd.concat([items_spend.drop('price', axis=1), fill_precio], axis=1)
items_spend.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3246362 entries, 0 to 3246361
Data columns (total 4 columns):
 #   Column       Dtype  
---  ------       -----  
 0   items_count  int64  
 1   user_id      object 
 2   item_id      int64  
 3   price        float64
dtypes: float64(1), int64(2), object(1)
memory usage: 99.1+ MB


In [39]:
items_spend.head()

,items_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,30,4.99
2,277,76561197970982479,300,9.99
3,277,76561197970982479,240,19.99
4,277,76561197970982479,3830,9.99


In [40]:
items_spend.isna().sum().sort_values(ascending= False)/len(items) * 100

items_count    0.0
user_id        0.0
item_id        0.0
price          0.0
dtype: float64

In [41]:
# Elimino 'item_id'. Ya lo use para unir con precio_juego. No lo necesito más
items_spend = items_spend.drop('item_id', axis=1)

In [42]:
# Agrupo por 'user_id' y sumo los precios de los items comprados por cada usuario
user_item_spend = items_spend.groupby('user_id')['price'].sum().reset_index()
user_item_spend

,user_id,price
0,--000--,182.84
1,--ace--,122.89
2,--ionex--,99.93
3,-2SV-vuLB-Kg,234.69
4,-404PageNotFound-,1154.47
...,...,...
68398,zzonci,0.00
68399,zzoptimuszz,4.99
68400,zzydrax,99.94
68401,zzyfo,484.73


In [43]:
user_item_count = items_spend[['items_count', 'user_id']]

# se eliminan los duplicados
user_item_count = user_item_count.drop_duplicates(subset='user_id', keep='first')
user_item_count

,items_count,user_id
0,277,76561197970982479
198,888,js41637
717,137,evcentric
821,328,Riot-Punch
951,541,doctr
...,...,...
3246347,5,76561198320038728
3246352,321,76561198320136420
3246354,4,ArkPlays7
3246356,22,76561198323066619


In [44]:
#uno user_item_count con user_item_spend y me queda la tabla final
items_spend = user_item_count.merge(user_item_spend, on='user_id', how='right')
items_spend

,items_count,user_id,price
0,58,--000--,182.84
1,44,--ace--,122.89
2,23,--ionex--,99.93
3,68,-2SV-vuLB-Kg,234.69
4,149,-404PageNotFound-,1154.47
...,...,...,...
68398,5,zzonci,0.00
68399,61,zzoptimuszz,4.99
68400,13,zzydrax,99.94
68401,84,zzyfo,484.73


A continuacion, creo un dataframe que contenga el tiempo jugado por usuario, por genero

In [45]:
items.columns

Index(['item_id', 'item_name', 'playtime_forever', 'steam_id', 'items_count',
       'user_id'],
      dtype='object')

In [46]:
user_time = items[['playtime_forever', 'user_id', 'item_id']]
user_time

,playtime_forever,user_id,item_id
0,0.10,76561197970982479,10
1,0.12,76561197970982479,30
2,78.88,76561197970982479,300
3,30.88,76561197970982479,240
4,5.55,76561197970982479,3830
...,...,...,...
3246357,11.28,76561198329548331,304930
3246358,0.72,76561198329548331,227940
3246359,0.05,76561198329548331,388490
3246360,0.07,76561198329548331,521570


In [47]:
# extraigo el genero de df games y cambio el nombre de la columna id por item_id para poder unilrla con user_time
item_genre = games[['genres', 'id']]
item_genre = item_genre.rename(columns={'id':'item_id'})
item_genre

,genres,item_id
0,Action,761140
1,Casual,761140
2,Indie,761140
3,Simulation,761140
4,Strategy,761140
...,...,...
71039,Indie,610660
71040,Racing,610660
71041,Simulation,610660
71042,Casual,658870


In [48]:
# uno user_time con item_genre con key: item id
genre_timeplay = user_time.merge(item_genre, on='item_id')
genre_timeplay = genre_timeplay.drop_duplicates(subset='user_id')

genre_timeplay

,playtime_forever,user_id,item_id,genres
0,0.10,76561197970982479,10,Action
323,3.67,js41637,300,Action
1485,15.38,evcentric,1200,Action
1723,1.12,Riot-Punch,300,Action
1934,18.85,doctr,300,Action
...,...,...,...,...
6805806,0.63,76561198320038728,304930,Action
6805824,1.72,76561198320136420,273350,Action
6805826,75.95,ArkPlays7,730,Action
6805832,0.13,76561198323066619,417860,Adventure


In [49]:
# se crea un nuevo dataframe con la suma de los tiempos de juego por género y usuario
genre_user_playtime = genre_timeplay.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()
genre_user_playtime

,genres,user_id,playtime_forever
0,Action,--ionex--,1.75
1,Action,-2SV-vuLB-Kg,1.78
2,Action,-404PageNotFound-,66.05
3,Action,-AnimeIsMyThing-,101.98
4,Action,-Azsael-,24.27
...,...,...,...
67843,Utilities,76561198100513560,0.02
67844,Utilities,76561198216710301,1.97
67845,Utilities,76561198262719220,0.02
67846,Utilities,ZertixCSGO,27.15


Creo un df que contenga el tiempo total jugado de por juego, genero y usuario, por año de lanzamiento

In [50]:
item_year_release = games[['id', 'release_year']]
# cambio el nombre de la columna id por item_id para poder unilrlla con genre_user_playtime
item_year_release = item_year_release.rename(columns={'id':'item_id'})
item_year_release

,item_id,release_year
0,761140,2018
1,761140,2018
2,761140,2018
3,761140,2018
4,761140,2018
...,...,...
71039,610660,2018
71040,610660,2018
71041,610660,2018
71042,658870,2017


In [51]:
genre_timeplay

,playtime_forever,user_id,item_id,genres
0,0.10,76561197970982479,10,Action
323,3.67,js41637,300,Action
1485,15.38,evcentric,1200,Action
1723,1.12,Riot-Punch,300,Action
1934,18.85,doctr,300,Action
...,...,...,...,...
6805806,0.63,76561198320038728,304930,Action
6805824,1.72,76561198320136420,273350,Action
6805826,75.95,ArkPlays7,730,Action
6805832,0.13,76561198323066619,417860,Adventure


In [52]:
# uno genre_timeplay con item_year_release con key: item id
usertime_year = genre_timeplay.merge(item_year_release,how='left', on='item_id')
usertime_year = usertime_year.drop_duplicates()
usertime_year

,playtime_forever,user_id,item_id,genres,release_year
0,0.10,76561197970982479,10,Action,2000
1,3.67,js41637,300,Action,2010
2,15.38,evcentric,1200,Action,2006
3,1.12,Riot-Punch,300,Action,2010
4,18.85,doctr,300,Action,2010
...,...,...,...,...,...
113486,0.63,76561198320038728,304930,Action,2017
113491,1.72,76561198320136420,273350,Action,2015
113493,75.95,ArkPlays7,730,Action,2012
113494,0.13,76561198323066619,417860,Adventure,2015


#### Creo DF que me facilite la consulta: cantidad de items y contenido free por desarrolladora por anio de lanzamiento

In [53]:
dev_price = games[['price', 'release_year', 'developer', 'id']]

#cambio el nombre de la columna id por item_id para poder unir con items_dev
dev_price = dev_price.rename(columns={'id':'item_id'})

# se eliminan los duplicados
items_dev = dev_price.drop_duplicates()

In [54]:
items_dev.head()

,price,release_year,developer,item_id
0,4.99,2018,Kotoshiro,761140
5,0.00,2018,Secret Level SRL,643980
9,0.00,2017,Poolians.com,670290
14,0.99,2017,彼岸领域,767400
17,3.99,2018,Trickjump Games Ltd,772540


Creo el DF best_dev según 'reviews_recommend' y 'sentiment_analysis'

In [55]:
item_sentiment = reviews[['reviews_item_id','reviews_recommend','sentiment_analysis']]

# renombro columna reviews_item_id a item_id para poder unir con items_developer
item_sentiment = item_sentiment.rename(columns={'reviews_item_id':'item_id'})

#uno items_developer con sentimiento_item con key: item id
best_dev = items_dev.merge(item_sentiment,how='left', on='item_id')

# Elimino los devs que no tienen item con review
best_dev = best_dev.dropna(subset=['reviews_recommend'])

# elimino los duplicados
best_dev = best_dev.drop_duplicates()

In [56]:
best_dev

,price,release_year,developer,item_id,reviews_recommend,sentiment_analysis
21,9.99,1997,Stainless Games Ltd,282010,True,1.0
22,9.99,1998,Valve,70,True,1.0
26,9.99,1998,Valve,70,True,0.0
28,9.99,1998,Valve,70,True,2.0
58,9.99,1998,Valve,70,False,2.0
...,...,...,...,...,...,...
74435,9.99,2004,Valve,80,True,1.0
74436,9.99,2004,Valve,80,True,2.0
74440,9.99,2004,Valve,80,True,0.0
74441,9.99,2004,Valve,80,False,1.0


In [57]:
#valores unicos de la columna sentiment_analysis
best_dev['sentiment_analysis'].unique()

array([1., 0., 2.])

In [58]:
#tipo de valor de la columna sentiment_analysis
best_dev['sentiment_analysis'].dtypes

dtype('float64')

In [59]:
#tipo de valor de la columna release_year
best_dev['release_year'].dtypes

dtype('int64')

Agrego año de lanzamiento al df Reviews

In [108]:
reviews

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,posted_year,sentiment_analysis,release_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011,1,2009
1,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,2011,2,2013
2,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,2013,2,2013
3,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,2014,1,2012
4,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,2014,0,2013
...,...,...,...,...,...,...,...,...
48685,kushikushigani,http://steamcommunity.com/id/kushikushigani,332310,No ratings yet,True,2015,1,2017
48686,How51,http://steamcommunity.com/id/How51,440,No ratings yet,True,2014,1,2007
48687,76561198111410893,http://steamcommunity.com/profiles/76561198111...,304930,No ratings yet,True,2014,1,2017
48688,zaza147,http://steamcommunity.com/id/zaza147,265630,No ratings yet,True,2015,2,2014


In [ ]:
item_year_release = games[['id', 'release_year']]

# Se renombra la columna 'id' para unirla con el dataframe anterior
item_year_release = item_year_release.rename(columns={'id':'reviews_item_id'})

# se eliminan los duplicados
item_year_release = item_year_release.drop_duplicates()


# Se une el lanzamiento con el item de reviews
reviews = reviews.merge(item_year_release, on='reviews_item_id')
reviews

In [109]:
#me fijo si en la columna recommend hay valores False
reviews['reviews_recommend'].value_counts()

reviews_recommend
True     43537
False     5153
Name: count, dtype: int64

In [41]:
reviews

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,posted_year,sentiment_analysis,release_year
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011,1,2009
1,evcentric,http://steamcommunity.com/id/evcentric,248820,No ratings yet,True,2011,2,2013
2,doctr,http://steamcommunity.com/id/doctr,250320,2 of 2 people (100%) found this review helpful,True,2013,2,2013
3,maplemage,http://steamcommunity.com/id/maplemage,211420,35 of 43 people (81%) found this review helpful,True,2014,1,2012
4,Wackky,http://steamcommunity.com/id/Wackky,249130,7 of 8 people (88%) found this review helpful,True,2014,0,2013
...,...,...,...,...,...,...,...,...
48685,kushikushigani,http://steamcommunity.com/id/kushikushigani,332310,No ratings yet,True,2015,1,2017
48686,How51,http://steamcommunity.com/id/How51,440,No ratings yet,True,2014,1,2007
48687,76561198111410893,http://steamcommunity.com/profiles/76561198111...,304930,No ratings yet,True,2014,1,2017
48688,zaza147,http://steamcommunity.com/id/zaza147,265630,No ratings yet,True,2015,2,2014


In [42]:
dfs = [reviews, items_spend, genre_timeplay, items_dev, usertime_year, best_dev]
# Nombres correspondientes a cada DataFrame
names = ['2-reviews', '4-items_spend', '6-genre_timeplay', '7-items_dev', '8-usertime_year', '9-best_dev']

for df, name in zip(dfs, names):
    archivo = f'data_clean/{name}.csv'
    df.to_csv(archivo, index=False, encoding='utf-8')

In [43]:
for df, name in zip(dfs, names):
    archivo = f'data_clean/{name}.parquet'
    pq.write_table(pa.Table.from_pandas(df), archivo)